In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import numpy as np
import keras
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [ ]:
paths = m21.corpus.getComposer('bach')
paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = utils.load_dataset_lstm(paths, 64, mapping)

print(f"Songs encoded: {len(paths)}")

In [ ]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

logdir = "logs/bach_lstm"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, tensorboard_callback=tensorboard_callback, epochs=10)

In [ ]:
model.save("Bach_soloMelodie_lstm")

In [ ]:
# Show the logs
%tensorboard --logdir logs/bach_lstm

In [ ]:
# Create song
generated_song = model.generate("n72 _ _ _ _ _ n72 _ _ _ _ _ n72 _ n71 _", max_length=500)    
new_song = utils.decode_songs_old(generated_song)

In [ ]:
path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)

In [ ]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}

# Train with new Dataformat from chinese dude

In [ ]:
# Load songs
paths = m21.corpus.getComposer('bach')
paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single_from_corpus(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.extend(encoded_song)

In [ ]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

logdir = "logs/bach_lstm_polyphonie"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, epochs=5, tensorboard_callback=tensorboard_callback)

In [ ]:
# Save model
model.save("Bach_polyphonie_lstm")

In [ ]:
# Show the logs
%tensorboard --logdir logs/bach_lstm_polyphonie

In [ ]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep")    

new_song_ints = mapping.numericalize(generated_song)
new_song = utils.from_polyphonic_encoding(np.array(new_song_ints), mapping, bpm=100)

path = Path("generated_song_bach_lstm.mid")
utils.write_midi(new_song, path)
